# Model ABC
## Create condition-specific EV files

In [7]:
import os
from os.path import join
import numpy as np
import pandas as pd
from glob import glob

subs = ['WMAZE_001', 'WMAZE_002', 'WMAZE_004', 'WMAZE_005', 'WMAZE_006',
        'WMAZE_007', 'WMAZE_008', 'WMAZE_009', 'WMAZE_010', 'WMAZE_012',
        'WMAZE_017', 'WMAZE_018', 'WMAZE_019', 'WMAZE_020', 'WMAZE_021',
        'WMAZE_022', 'WMAZE_023', 'WMAZE_024', 'WMAZE_026', 'WMAZE_027']
stim_sets = ['set1', 'set2', 'set3']

for sub in subs:
    sub_dir = '/home/data/madlab/data/mri/wmaze/scanner_behav/{0}/'.format(sub)
    dir_file = glob(join(sub_dir, '{0}_wmazebl_2015*.txt'.format(sub)))   
    dir_file.sort() 
    for i, curr_set in enumerate(stim_sets):
        for curr_run in ['1', '2']:
            if curr_run == '1':
                run = pd.read_table(dir_file[i * 2]) #create dataframe for text files to extract EVS
                run = run[:-3] #removal of the last 3 trials to avoid scanner artifact
            else:
                run = pd.read_table(dir_file[i * 2 + 1])
                run = run[:-3]
        
            trial = run['TrialType'].values #convert dataframes into numpy arrays
            correct = run['Correct'].values
            onsets = run['StimOnset'].values
                       
            #grab indices matching multiple specified criteria
            A_corr = np.where((trial == 'A') & (correct == 1))[0]
            B_corr = np.where((trial == 'B') & (correct == 1))[0]
            C_corr = np.where((trial == 'C') & (correct == 1))[0]
            A_incorr = np.where((trial == 'A') & (correct == 0))[0]
            B_incorr = np.where((trial == 'B') & (correct == 0))[0]
            C_incorr = np.where((trial == 'C') & (correct == 0))[0]            
            BL = np.where((trial == 'BL'))[0]

            #index onsets array using indices from np.where() criteria 
            A_corr_onsets = onsets[A_corr] 
            B_corr_onsets = onsets[B_corr] 
            C_corr_onsets = onsets[C_corr] 
            A_incorr_onsets = onsets[A_incorr] 
            B_incorr_onsets = onsets[B_incorr] 
            C_incorr_onsets = onsets[C_incorr]
            BL_onsets = onsets[BL]
       
            #v-stack matrix containing *ALL* onsets, durations, and amplitudes in vertical columns 
            mtrx = np.vstack((onsets, np.ones(len(onsets))*2.5, #numpy array filled with 3's
                              np.ones(len(onsets)))).T #numpy array filled with 1's
            A_corr_mtrx = np.vstack((A_corr_onsets, np.ones(len(A_corr_onsets))*2.5,
                                     np.ones(len(A_corr_onsets)))).T 
            B_corr_mtrx = np.vstack((B_corr_onsets, np.ones(len(B_corr_onsets))*2.5,
                                     np.ones(len(B_corr_onsets)))).T 
            C_corr_mtrx = np.vstack((C_corr_onsets, np.ones(len(C_corr_onsets))*2.5,
                                     np.ones(len(C_corr_onsets)))).T 
            A_incorr_mtrx = np.vstack((A_incorr_onsets, np.ones(len(A_incorr_onsets))*2.5,
                                      np.ones(len(A_incorr_onsets)))).T 
            B_incorr_mtrx = np.vstack((B_incorr_onsets, np.ones(len(B_incorr_onsets))*2.5,
                                     np.ones(len(B_incorr_onsets)))).T 
            C_incorr_mtrx = np.vstack((C_incorr_onsets, np.ones(len(C_incorr_onsets))*2.5,
                                     np.ones(len(C_incorr_onsets)))).T 
            BL_mtrx = np.vstack((BL_onsets, np.ones(len(BL_onsets))*2.5, np.ones(len(BL_onsets)))).T
                        
            if not os.path.exists(join(sub_dir, 'model_ABC/')): #if directory does not exist
                os.makedirs(join(sub_dir, 'model_ABC/')) #create it

            if curr_run == '1': #if the first run in a stimulus set
                np.savetxt(sub_dir+'model_ABC/'+'run{0}.txt'.format(i*2+1),mtrx,delimiter='\t',fmt='%.4f')                
                for trial in ['A_corr', 'B_corr', 'C_corr', 'A_incorr', 'B_incorr', 'C_incorr', 'BL']: #for all trial types
                    exec('np.savetxt(sub_dir+"model_ABC/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+1,trial))

            else: #if the second run in a stimulus set
                np.savetxt(sub_dir+'model_ABC/'+'run{0}.txt'.format(i*2+2),mtrx,delimiter='\t',fmt='%.4f')                
                for trial in ['A_corr', 'B_corr', 'C_corr', 'A_incorr', 'B_incorr', 'C_incorr', 'BL']:
                    exec('np.savetxt(sub_dir+"model_ABC/"+"run{0}_{1}.txt",{1}_mtrx,delimiter="\t",fmt="%.4f")'.format(i*2+2,trial))